# 数据清洗

数据清洗策略子项:
- 申请者个人 / 家庭基本信息
- 申请者联系方式
- 申请者车辆购置情况
- 申请者工作情况
- 申请者房产情况
- 贷款申请材料
- 申请者社交状况
- 风险评估

每个策略子项从两个方面进行清洗:
- 与业务逻辑相结合判断是否存在异常值
- 从统计学意义上进行判断是否存在异常值

清洗完成后:
- **标记**异常项, 暂时不删除异常项
- 对文本类型的特征项进行数值化处理
- 对数值化类型的特征项进行合适的数值化处理
- 对于缺失值填充`np.nan`或者单独列成一类

In [182]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [183]:
DATA_DIR = '../data'
APP_TRAIN_FILENAME = 'application_train.csv'
APP_TEST_FILENAME = 'application_test.csv'
BUREAU_FILENAME = 'bereau.csv'
BUREAU_BALANCE_FILENAME = 'bureau_balance.csv'
CREDIT_CARD_BALANCE_FILENAME = 'credit_card_balance.csv'
INSTALLMENTS_PAYMENTS = 'installments_payments.csv'
POS_CACHE_BALANCE_FILENAME = 'POS_CACHE_balance.csv'
PREVIOUS_APP_FILENAME = 'previous_application.csv'

In [184]:
user_df = pd.read_csv(os.path.join(DATA_DIR, APP_TRAIN_FILENAME))
cleaned_df = user_df.loc[:, ['SK_ID_CURR', 'TARGET']].copy()

In [185]:
def detectOutlier(dataSeries):
    quantile_1_4 = dataSeries.quantile(0.25)
    quantile_3_4 = dataSeries.quantile(0.75)
    validRange = 1.5 * (quantile_3_4 - quantile_1_4)
    minValue = quantile_1_4 - validRange
    maxValue = quantile_3_4 + validRange
    return dataSeries[(dataSeries > maxValue) | (dataSeries) < minValue]

In [186]:
def detectOutlierRecords(dataFrame, cols):
    outlierCounter = np.full((dataFrame.shape[0], ), 0, dtype=np.int)
    for col in cols:
        curOutlierFlag = detectOutlier(dataFrame[col])
        print(col, curOutlierFlag)
        outlierCounter[curOutlierFlag] = outlierCounter[curOutlierFlag] + 1
    return outlierCounter

## 申请者个人 / 家庭基本信息

主要包含以下字段:
- SK_ID_CURR
- TARGET
- CODE_GENDER
- DAYS_BIRTH
- DAYS_REGISTRATION
- DAYS_ID_PUBLISH
- NAME_EDUCATION_TYPE
- CNT_CHILDREN
- CNT_FAM_MEMBERS
- NAME_FAMILY_STATUS

有效性检查包括:

- DAYS_BIRTH >= DAYS_REGISTRATION
- ~~DAYS_BIRTH >= DAYS_ID_PUBLISH~~
- ~~NAME_FAMILY_STATUS 是否与 CNT_CHILDREN 以及 CNT_FAM_MEMBERS 冲突~~
- ~~CNT_FAM_MEMBERS < CNT_CHILDREN~~

**ATTN**:
- 对于原表使用`loc`选择单列是引用, 选择多列是复制.

In [187]:
PER_FAM_FACTORS = ['SK_ID_CURR', 'TARGET', 'CODE_GENDER',
                   'DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
                   'NAME_EDUCATION_TYPE', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS',
                   'NAME_FAMILY_STATUS']

In [188]:
print('-' * 100)
print(user_df.loc[user_df['DAYS_BIRTH'] > user_df['DAYS_REGISTRATION'], ['SK_ID_CURR', 'TARGET', 'DAYS_BIRTH', 'DAYS_REGISTRATION']])
print('-' * 100)
print(user_df.loc[user_df['DAYS_BIRTH'] > user_df['DAYS_ID_PUBLISH'], ['SK_ID_CURR', 'TARGET', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH']])
print('-' * 100)
print(user_df.loc[user_df['CNT_FAM_MEMBERS'] < user_df['CNT_CHILDREN'], ['SK_ID_CURR', 'TARGET', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN']])
print('-' * 100)

----------------------------------------------------------------------------------------------------
        SK_ID_CURR  TARGET  DAYS_BIRTH  DAYS_REGISTRATION
266366      408583       0      -10116      -10116.041667
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [SK_ID_CURR, TARGET, DAYS_BIRTH, DAYS_ID_PUBLISH]
Index: []
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [SK_ID_CURR, TARGET, CNT_FAM_MEMBERS, CNT_CHILDREN]
Index: []
----------------------------------------------------------------------------------------------------


### CODE_GENDER

- 'M' - 0
- 'F' - 1
- 'XNA' - `np.nan`

In [189]:
user_df.loc[:, 'CODE_GENDER'].unique()

array(['M', 'F', 'XNA'], dtype=object)

In [190]:
cleaned_df.loc[:, 'CODE_GENDER'] = user_df['CODE_GENDER'].replace(['M', 'F', 'XNA'], [0, 1, np.nan])

### DAYS_BIRTH

In [191]:
cleaned_df.loc[:, 'DAYS_BIRTH'] = -user_df['DAYS_BIRTH']

### DAYS_REGISTRATION

In [192]:
cleaned_df.loc[:, 'DAYS_REGISTRATION'] = -user_df['DAYS_REGISTRATION']

### DAYS_ID_PUBLISH

In [193]:
cleaned_df.loc[:, 'DAYS_ID_PUBLISH'] = -user_df['DAYS_ID_PUBLISH']

### NAME_EDUCATION_TYPE

- `Secondary / secondary special` - 0
- `Higher education` - 1
- `Incomplete higher` - 2
- `Lower secondary` - 3
- `Academic degree` - 4

In [194]:
cleaned_df.loc[:, 'NAME_EDUCATION_TYPE'] = user_df['NAME_EDUCATION_TYPE'].replace(['Secondary / secondary special', 'Higher education', 'Incomplete higher', 'Lower secondary', 'Academic degree'], range(0, 5))

### CNT_CHILDREN

In [195]:
cleaned_df.loc[:, 'CNT_CHILDREN'] = user_df['CNT_CHILDREN']

### CNT_FAM_MEMBERS

In [196]:
cleaned_df.loc[:, 'CNT_FAM_MEMBERS'] = user_df['CNT_FAM_MEMBERS'].fillna(np.nan)

### NAME_FAMILY_STATUS

- Married 0
- Single / not married 1
- Civil marriage 2
- Separated 3
- Widow 4
- Unknown 5

In [197]:
cleaned_df.loc[:, 'NAME_FAMILY_STATUS'] = user_df['NAME_FAMILY_STATUS'].replace(['Married', 'Single / not married', 'Civil marriage', 'Separated', 'Widow', 'Unknown'], [0, 1, 2, 3, 4, np.nan])

## 申请者联系方式

- FLAG_MOBIL
- FLAG_PHONE
- FLAG_EMAIL
- FLAG_CONT_MOBILE

In [198]:
CONTACT_FACTOR = ['FLAG_MOBIL', 'FLAG_PHONE', 'FLAG_EMAIL', 'FLAG_CONT_MOBILE']

### FLAG_MOBIL

In [199]:
cleaned_df.loc[:, 'FLAG_MOBIL'] = user_df['FLAG_MOBIL']

### FLAG_PHONE

In [200]:
cleaned_df.loc[:, 'FLAG_PHONE'] = user_df['FLAG_PHONE']

### FLAG_EMAIL

In [201]:
cleaned_df.loc[:, 'FLAG_EMAIL'] = user_df['FLAG_EMAIL']

### FLAG_CONT_MOBILE

In [202]:
cleaned_df.loc[:, 'FLAG_CONT_MOBILE'] = user_df['FLAG_CONT_MOBILE']

## 申请者车辆购置情况

- FLAG_OWN_CAR
- OWN_CAR_AGE

### FLAG_OWN_CAR

- N - 0
- Y - 1

In [203]:
cleaned_df.loc[:, 'FLAG_OWN_CAR'] = user_df['FLAG_OWN_CAR'].replace(['N', 'Y'], [0, 1])

### OWN_CAR_AGE

In [204]:
cleaned_df.loc[:, 'OWN_CAR_AGE'] = user_df['OWN_CAR_AGE'].fillna(np.nan)

## 申请者工作情况

- DAYS_EMPLOYED
- FLAG_EMP_PHONE
- FLAG_WORK_PHONE
- ORGANIZATION_TYPE
- NAME_INCOME_TYPE
- OCCUPATION_TYPE

### DAYS_EMPLOYED

In [205]:
cleaned_df.loc[:, 'DAYS_EMPLOYED'] = -(user_df['DAYS_EMPLOYED'].replace(365243, np.nan))

### FLAG_EMP_PHONE

In [206]:
cleaned_df.loc[:, 'FLAG_EMP_PHONE'] = user_df['FLAG_EMP_PHONE']

### FLAG_WORK_PHONE

In [207]:
cleaned_df.loc[:, 'FLAG_WORK_PHONE'] = user_df['FLAG_WORK_PHONE']

### ORGANIZATION_TYPE

没有缺失值, 直接映射成数值.

In [208]:
cleaned_df.loc[:, 'ORGANIZATION_TYPE'] = user_df['ORGANIZATION_TYPE'].replace(user_df['ORGANIZATION_TYPE'].unique(), range(len(user_df['ORGANIZATION_TYPE'].unique())))

### NAME_INCOME_TYPE

没有缺失值, 直接映射成数值.

In [209]:
cleaned_df.loc[:, 'NAME_INCOME_TYPE'] = user_df['NAME_INCOME_TYPE'].replace(user_df['NAME_INCOME_TYPE'].unique(), range(len(user_df['NAME_INCOME_TYPE'].unique())))

### OCCUPATION_TYPE

In [220]:
occupationType = user_df.loc[user_df['OCCUPATION_TYPE'].notnull(), 'OCCUPATION_TYPE'].unique()

In [223]:
cleaned_df.loc[:, 'OCCUPATION_TYPE'] = user_df['OCCUPATION_TYPE'].replace(occupationType, range(len(occupationType)))